In [ ]:
# import macro-functions (libraries) to perform high level computation and funtions

import pandas as pd   # to manage table

import numpy as np    # to manage linear algebra

import matplotlib.pyplot as plt      # to plot data

import seaborn as sns                # to plot data
#sns.set_theme();

import statsmodels.api as sm         # to use statistical tools

from sklearn.preprocessing import scale   # tools to preprocess data

import warnings; warnings.filterwarnings('ignore') # "default" restore default mode "ignore" ignore

## **SCENARIO**: You are working in the marketing department of a car seller. 
## Your boss gives you the latest sales data and assigns you the following tasks:
1) To create a summary report on the total sales for each Manufacturer
2) To identify what factors influence the sales
3) To make a prediction about how many sales can be generated with an investment of 500k euro in TV Advertising
4) To make a prediction about how many sales can be generated with an investment of 75k euro in Social Advertising
5) To make a prediction for some new cars models about a chance to win the prestigious award "Car of the Year". Factors of success are Power performance and Fuel Efficiency. Data about these factors for the new cars will be provided by marketing dept.

All data are in excel files provided by IT department. 

# Task 1: create a summary report on the total sales for each Manufacturer

First I load all the excel file directly in a table

In [ ]:
# ANACONDA NAVIGATOR -- UNCOMMENT HERE
#sales_table = pd.read_csv('./Data/sales_data.csv') # Load a CSV file on a local table (dataframe)

# GOOGLE COLAB - UNCOMMENT HERE

url = 'https://raw.githubusercontent.com/pal-dev-labs/Python-for-Economic-Applications-2024-2025/refs/heads/main/Data/sales_data.csv'
sales_table = pd.read_csv(url)

In [ ]:
# print the dataframe info

sales_table.info()

In [ ]:
# print the dataframe content

sales_table

## DATA EXPLORATION

First let's see how many different models has each Manifacturer 

In [ ]:
# for each manufacturer group the models and plot a barplot
sales_table['Manufacturer'].value_counts().plot.bar();

Let's save this image for the future report. I need a nicer chart

In [ ]:
sales_table['Manufacturer'].value_counts().plot.bar()

# add some decorators
plt.xlabel('Manufacturers')
plt.ylabel('Number of Models')
plt.title('Manufacturers different models')
plt.legend()
plt.savefig('manufacturer.png')  # this saves the figure i

Let's extract total amount of sales for each manufacturer. I use a **pivot table**

In [ ]:
total_sales = pd.pivot_table(sales_table, index=['Manufacturer'], values=['Sales_in_thousands'],aggfunc=[np.sum])
total_sales

Let's order a little bit

In [ ]:
total_sales = total_sales.sort_values(by=('sum', 'Sales_in_thousands'), ascending=False)
total_sales

In [ ]:
total_sales.plot.bar(legend = False)
plt.ylabel('Sales (€)');
plt.xlabel('Manufacturers')
plt.title('Summary of Manufacturers total sales')
plt.legend()
plt.savefig('manufacturer_total_sales.png')  # this saves the figure i

In [ ]:
total_sales.iloc[0:15].plot.pie(subplots=True, legend= False, autopct="%1.1f%%")
plt.ylabel('');
plt.xlabel('')
plt.title('Manufacturers Total Sales (k€)')
plt.savefig('manufacturer_total_sales2.png') 

# Task 2: identify what factors influence the sales

### Let's see the the role of the price

In [ ]:
plt.scatter(sales_table['Price_in_thousands'].values, sales_table['Sales_in_thousands'].values)
plt.xlabel("Price_in_thousands");plt.ylabel("Sales_in_thousands");

## Let's try with more features

In [ ]:
g = sns.pairplot(sales_table.iloc[:,[3,4,5,6,7,8]]);
g.fig.suptitle("Factors that could influence the sales", fontsize=25)
plt.show()

## Price, TV Advertising (very correlated) seems interesting

## Let's try to calculate a correlation 

In [ ]:
cor_tv = sales_table['Sales_in_thousands'].corr(sales_table['TV Advert (thousands)'])
cor_price = sales_table['Sales_in_thousands'].corr(sales_table['Price_in_thousands'])
print("Correlation between Sales and TV Advertising:", cor_tv)
print("Correlation between Sales and Price:", cor_price)

## TV Advertising seems to be very linearly correlated with Sales.

## Could be interesting to perform a linear regression to have a predictor for sales

$ Sales = \beta_0 + \beta_1 (TVAdv) + \epsilon$

$ y = X \beta + \epsilon$

In [ ]:
# we use statsmodels mudule as sm for statistical tools


# extract X training data from dataframe
X_train = sales_table['TV Advert (thousands)'].values
X_train = sm.add_constant(X_train)

# extract y training data from dataframe
y_train = sales_table['Sales_in_thousands'].values


In [ ]:
X_train[0:20]

In [ ]:
# set simple linear regression model
model = sm.OLS(y_train,X_train)    # create the OLS model

# train the model
results= model.fit()   # train the model

# Print the model summary
print(results.summary())

In [ ]:
print("Coefficients: ", results.params)
print("R2: ", results.rsquared)

## Let's try to visualize the regression line 

In [ ]:
# ------------------------------------
# calculate predictions
pred_ols = results.get_prediction()

#-------------------------------------
# Confidence intervals
iv_l = pred_ols.summary_frame()["obs_ci_lower"]
iv_u = pred_ols.summary_frame()["obs_ci_upper"]

#--------------------------------------
# plotting
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(X_train[:,1], y_train, "o", label="data")   # plot original data
ax.plot(X_train[:,1], results.fittedvalues, "r--.", label="OLS")  # plot fitted data
ax.plot(X_train[:,1], iv_u, "r--", c='g')   # plot upper confident interval
ax.plot(X_train[:,1], iv_l, "r--", c='g')   # plot lower confident interval

#---------------------------------------
# figure decorator
ax.legend(loc="best")
plt.xlim(0,600)
plt.ylim(0,70)
plt.xlabel('TV Advertising (k euros)')
plt.ylabel('Sales (k)')
plt.show()

# TASK 3: 
## Make a prediction about how many sales can be generated with an investment of 500k euro in TV Advertising

## Having a statistical model for TV Advertising we can estimate sales 

In [ ]:
inv = 500
β1 = results.params[0]  # estimated parameter
β2 = results.params[1]  # estimated parameter
y_pred = β1 + β2 * inv
print("Estimated amount of sales with investment of",inv,"k euros is",y_pred, "k units")

# TASK 4: 
## Make a prediction about how many sales can be generated with an investment of 75k euro in Social Advertising

### Let's picture the relationship betweeen Social Adv and Sales

In [ ]:
plt.scatter(sales_table['Social Advert'].values, sales_table['Sales_in_thousands'].values)
plt.xlabel("Social Advert")
plt.ylabel("Sales_in_thousands");


It looks not properly linear...

## Let's try to calculate a correlation 

In [ ]:
cor_sol = sales_table['Sales_in_thousands'].corr(sales_table['Social Advert'])
print("Correlation between Sales and Social Advertising:", cor_sol)


It's very poor....

Let's try with the statistical tool for linear regressio

In [ ]:
X_train_social = sales_table['Social Advert'].values

In [ ]:
# train the model

model1 = sm.OLS(y_train,X_train_social)    # create the OLS model
results1 = model1.fit()   # train the model

# Print the model summary
print(results1.summary())

## We got poor results. 
## Let's try with a different (non linear) estimator: 

# k-nearest neighbors 
(https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)  
In statistics, the **k-nearest neighbors algorithm (k-NN)** is a **non-parametric supervised learning method**.

The k-NN algorithm can also be generalized for **regression**. In k-NN regression, also known as **nearest neighbor smoothing**, the output is the property value for the object. This **value is the average of the values of k nearest neighbors**. If k = 1, then the output is simply assigned to the value of that single nearest neighbor, also known as nearest neighbor interpolation.

In python the algorithm is implemented in **sklearn** Module with the name of **KNeighborsRegressor** (https://scikit-learn.org/1.5/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)

In [ ]:
# extract data from original sales dataframe
X_social = sales_table['Social Advert'].values
y = sales_table['Sales_in_thousands'].values

In [ ]:
# import the libraries
from sklearn import neighbors   # to import the model


In [ ]:
# preprocess inputs for the model
from sklearn.preprocessing import scale   # to preprocess the input
X_social_prep = scale(X_social, with_mean=False, with_std=False)
X_social_prep = X_social_prep.reshape(-1,1)
X_social_prep[0:10]

The **scale function** from sklearn.preprocessing **standardizes the input data**, typically by **removing the mean and scaling to unit variance**. However, the parameters with_mean=False and with_std=False modify its behavior.  
- with_mean=False: The function does not center the data by subtracting the mean.
- with_std=False: The function does not scale the data by dividing by the standard deviation.
  
In this case, no actual scaling or centering is performed. The data will remain unchanged, but the call to scale maintains compatibility with a preprocessing pipeline.



### Let's split the data into a training set (used to fit the model) and a test set (to check the accuracy of the model)

In [ ]:

# split train test and test test to check accuracy of the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_social_prep, y, test_size=0.25)


### Model fitting (we have 1 hyperparameter)

In [ ]:
# set model hyperparameters
n_neighbors = 20

# set the model with hyperparameter
knn = neighbors.KNeighborsRegressor(n_neighbors)

# fit the model
results = knn.fit(X_train, y_train)

### Check the accuracy of the model with Mean Square Error (MSE)
If a vector of **$n$ predictions** is generated from a sample of **$n$ data points** on all variables, and $Y$ **is the vector of observed values** of the variable being predicted, with $\hat {Y}$ being the **predicted values** (e.g. as from a least-squares fit), then the within-sample MSE of the predictor is computed as

$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (Y_i - \hat{Y}_i)^2
$

### How to use the MSE?
We will compare the **training MSE** and **test MSE** in order to check if we have a good model.

### a) Compare the train and test MSE Values
- **Low MSE for both train and test sets** indicates that the **model fits well** and generalizes effectively.
- **Low MSE for train but high MSE for test** suggests **overfitting** (the model fits the training data too closely but fails to generalize).
- **High MSE for both train and test** indicates **underfitting** (the model is too simple to capture patterns in the data).


In [ ]:
# Calculate the mean squared error (MSE)
def mse(y_true, y_hat):
  return np.mean((y_true - y_hat) ** 2)/len(y_true)

# Calculate the predicted Y from train set
y_hat_train = results.predict(X_train)

# Calculate the train MSE
mse_value_train = mse(y_train, y_hat_train)

# Calculate the predicted Y from test set
y_hat_test = results.predict(X_test)

# Calculate the test MSE
mse_value_test = mse(y_test, y_hat_test)

# Print the MSE
print("MSE train:",mse_value_train)
print("MSE test:",mse_value_test)

## Note that if we repeat the shuffle between train and set the MSE change

Let's assume the values tell us that the model is accurate enough. Let's have a look of the prediction


In [ ]:
y_hat = results.predict(X_social.reshape(-1,1))

plt.scatter(X_social, y, color='darkorange', label='data')
plt.scatter(X_social, y_hat, color='navy', label='prediction')
plt.legend()
plt.title("KNeighborsRegressor (k = %i)" % (n_neighbors))

plt.tight_layout()
plt.xlim(0,200000)
plt.ylim(0,400)
plt.xlabel("Social Advertising investment (euros)")
plt.ylabel("Sales (k units)")

plt.show()

## Let's assume instead that the 2 values not good and our model not accurate

## Do we have the best model? We can change the hyperparameters **n_neighbors** and check MSE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_social_prep, y, test_size=0.25)

In [ ]:
# split train test and test test to check accuracy of the model
X_train, X_test, y_train, y_test = train_test_split(X_social_prep, y, test_size=0.25)

# we will use the SAME set to train the model with different hyperparameters
mse_value_train = []
mse_value_test = []

for i in range(1,100):
    n_neighbors = i
    knn = neighbors.KNeighborsRegressor(n_neighbors)
    results = knn.fit(X_train, y_train)
    # Calculate the test MSE
    y_hat_train = results.predict(X_train)
    mse_value_train.append(mse(y_train, y_hat_train))
    # Calculate the test MSE
    y_hat_test = results.predict(X_test)
    mse_value_test.append(mse(y_test, y_hat_test))

In [ ]:
plt.plot(mse_value_train, label='Train MSE')
plt.plot(mse_value_test, label='Test MSE')
plt.legend()
plt.xlabel('Neighbors')
plt.ylabel('MSE')

## we can take neighbors = 50 as best hyperparameter and make a prediction for the investment of 75k euro in Social Advertising


In [ ]:
## we can take neighbors = 50 as best hyperparameter and prediction about and make a prediction for the investment of 75k euro in Social Advertising
n_neighbors = 50

knn = neighbors.KNeighborsRegressor(n_neighbors)
results = knn.fit(X_train, y_train)
y_hat = results.predict(np.array([75000]).reshape(-1,1))

print('An investment of 75k euro in Social Advertising generates ',y_hat[0], 'units of sale')

# TASK 5: 
make a prediction for some new cars models about a chance to win the prestigious award "Car of the Year". Factors of success are Power performance and Fuel Efficiency. Data about these factors for the new cars will be provided

## Last question: The award

### We have to make a prediction for three new cars models about a chance to win the prestigious award "Car of the Year".

### Factors of success are Power performance and Fuel Efficiency. 

In [ ]:
sales_table[['Manufacturer','Model','Fuel_efficiency','Power_perf_factor','Awarded']]

### We note that the column "Awarded" has 0 for NOT win and 1 for WIN.

### We have to build a standard classifier with 2 predictors: "Fuel_efficiency" and "Power_perf_factor"

## Let's load the file provided by marketing dep containing the new cars 

In [ ]:
# Load the CSV file provided by IT on a local table (dataframe)
award_factors = pd.read_csv('https://raw.githubusercontent.com/pal-dev-labs/Python-for-Economic-Applications-2024-2025/refs/heads/main/Data/award-factors.csv')

In [ ]:
award_factors

## We will use a Neural Network as a classifier. We will use 2 preditors 'Fuel_efficiency' and 'Power_perf_factor'

In [ ]:
# Split the data into features and target
X = sales_table[['Fuel_efficiency', 'Power_perf_factor']]
y = sales_table['Awarded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
from tensorflow import keras

# Define the model
model = keras.Sequential([
  keras.layers.Dense(16, activation='relu', input_shape=(2,)),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model on the training set
model.fit(X_train, y_train, epochs=100);

In [ ]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the loss and accuracy
print('Loss:', loss)
print('Accuracy:', accuracy)

## Make predictions on new cars


In [ ]:
# Make predictions on new data

predictions = model.predict(award_factors[['Fuel_efficiency', 'Power_perf_factor']])
award_factors['Probability to Win (%)']=np.around(predictions*100,2)

award_factors